In [7]:
##Brute-Force, Exhaustive Search and Branch&Bound!

import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import scipy.stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression  
import itertools
import copy

In [117]:
## Get Data as a Pandas Dataframe! Make sure to have numerical data only! (pd.get_dummies might help)

file ="C:/Users/Praktikant1/Downloads/nuclear.csv"

data = pd.read_csv(file,sep=",").astype(float).drop(data.columns[1], axis=1)

#Y = list(data)[0]
#X = list(data)[1:]

print(data)

    Unnamed: 0    cost    t1    t2     cap   pr   ne   ct   bw  cum.n   pt
0          1.0  460.05  14.0  46.0   687.0  0.0  1.0  0.0  0.0   14.0  0.0
1          2.0  452.99  10.0  73.0  1065.0  0.0  0.0  1.0  0.0    1.0  0.0
2          3.0  443.22  10.0  85.0  1065.0  1.0  0.0  1.0  0.0    1.0  0.0
3          4.0  652.32  11.0  67.0  1065.0  0.0  1.0  1.0  0.0   12.0  0.0
4          5.0  642.23  11.0  78.0  1065.0  1.0  1.0  1.0  0.0   12.0  0.0
5          6.0  345.39  13.0  51.0   514.0  0.0  1.0  1.0  0.0    3.0  0.0
6          7.0  272.37  12.0  50.0   822.0  0.0  0.0  0.0  0.0    5.0  0.0
7          8.0  317.21  14.0  59.0   457.0  0.0  0.0  0.0  0.0    1.0  0.0
8          9.0  457.12  15.0  55.0   822.0  1.0  0.0  0.0  0.0    5.0  0.0
9         10.0  690.19  12.0  71.0   792.0  0.0  1.0  1.0  1.0    2.0  0.0
10        11.0  350.63  12.0  64.0   560.0  0.0  0.0  0.0  0.0    3.0  0.0
11        12.0  402.59  13.0  47.0   790.0  0.0  1.0  0.0  0.0    6.0  0.0
12        13.0  412.18  1

In [83]:
## exhaustive search - Brute Force. create class here

class BruteForce: 
    
    def __init__(self, Y, X, data):
        self.Y = Y
        self.X = X
        self.data = data
   
    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                #print(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(data[X],data[Y])  
        
        #returns R^2
        s = regressor.score(data[X], data[Y])
        
        # Calculates adjusted R^2
        n = len(data)
        k = len(self.X)
        adj_r2 = 1 - (( (1-r2)*(n-1) )/ (n - k -1))
        
        return (s) 
    
    def search(self):
        solutions = []
        combos = self.powerset(self.X)
        for i in combos:
            solutions.append(self.regress(i, self.Y, self.data))  
        return solutions, combos
    
    def solve(self):
        sol, combos = self.search()
        highest_R2 = combos[np.argmax(sol)]
        R2 = max(sol)
        
        print("higehst R^2 is", R2, "with dependant variables:", highest_R2)
        return highest_R2, R2
        
        

In [84]:
t, r2 = BruteForce(Y,X,data).solve()

print(len(list(data)))

higehst R^2 is 0.839423385282796 with dependant variables: ['date', 't1', 't2', 'cap', 'pr', 'ne', 'ct', 'bw', 'cum.n', 'pt']
11


In [118]:
## this will solve all possible linear regressions and return the one with highest R^2

class exhaustivesearch: 
    
    def __init__(self, data):
        self.data = data
        
    def assign(self):
        XX = list(self.data)
        dic = {}
        
        for i in XX:
            Y = i
            X = self.powerset([x for x in XX if x != Y])
            dic[Y] = X
        return dic

    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(self.data[X],self.data[Y])
        
        #returns R^2
        r2 = regressor.score(self.data[X], self.data[Y])
        
        #returns Adjusted R^2 
        n = len(self.data)
        k = len(list(self.data[X]))
        adj_r2 = (1 - ( ((1 - r2)*(n - 1))/(n - k - 1) ))
        
        return adj_r2 
    
    def search(self):
        dic = self.assign()
        newdic = {}
        for y in dic:
            for x in dic[y]:
                newdic[self.regress(x, y, data)] = y, x
        new={}
        sd = sorted(newdic.items(), reverse=True)
        for k,v in sd:
            new[k] = v
        return new
    
    def solve(self):
        results = self.search()
        highest_R2 = list(results.keys())[0]
        best_vars = results[highest_R2]
        
        print("higehst R^2 is", highest_R2, "with dependant/indepedant variables:", best_vars)
        return results

In [119]:
wow = exhaustivesearch(data).solve()

higehst R^2 is 0.8283937667274062 with dependant/indepedant variables: ('pt', ['Unnamed: 0', 'cost', 't1', 'cap', 'ne'])


In [124]:
## Next feature: restricting size, upper and lower bounds for model size!

# wow = exhaustive_search(data = data, R^2 = adj, exp_var = 5, lower_bound = 3, upper_bound = 7).solve()


In [121]:
## compute adjusted R2
def adj_r2(r2, data, X):
    n = len(data)
    k = len(X)
    adj_r2 = 1 - (( (1-r2)*(n-1) )/ (n - k -1))
    
    return adj_r2

In [123]:
len(data)

32